### Imports

In [ ]:
import os
import yaml
import polars as pl

### Experiment description

Example..

1. Predict ClusterID for clustering datasets
2. Use PointTransformer
3. Features = xy
4. Homogeneous graph

Change everywhere it says CHANGEME

### Variables

In [ ]:
input_folder = '../../../data/nieves'
output_folder = '../../../output/nieves/CHANGEME'
input_expt_folder = '../../../data/nieves/CHANGEME'

### Preprocess data to suitable output location

In [ ]:
folders = [output_folder, os.path.join(input_folder, "config/CHANGEME"), input_expt_folder]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
# Code specific to dataset & experiment 1
root_dir = os.path.join(input_folder, "ml_data")
for file in os.listdir(root_dir):
    df = pl.read_csv(os.path.join(root_dir, file))
        
    if file.startswith('gt_'):
        df = df.rename({'':'x', '_duplicated_0':'y', 'moleculeClusterIndex': 'gt_label'})
    elif file.startswith('multiple_blinking'):
        df = df.drop(['error', 'moleculeIndex'])
        df = df.rename({'clusterIndex': 'gt_label'})
        
    # change all cluster id's which are != 0 to 1 so we can predict not clustered or clustered
    df = df.with_columns((pl.when(pl.col('gt_label') != 0)).then(1).otherwise(0).alias('gt_label'))
    
    # save 
    file_name, _ = os.path.splitext(file)
    file_name = os.path.join(input_expt_folder, file_name + '.parquet')
    df.write_parquet(file_name)

In [ ]:
# Export config variables to file

config = {}

# The following are the names of the 
# x column, y column, z column if present, channel, frame,
# in the csvs being processed
config['x_col'] = 'x'
config['y_col'] = 'y'
config['z_col'] = None
config['channel_col'] = None #'channel'
config['frame_col'] = None # 'frame'

# The number of dimensions to consider
# If 2 only deals with x and y 
# If 3 will read in and deal with z as well (currently not fully supported)
config['dim'] = 2

# choice of which channels user wants to consider
# if null considers all
config['channel_choice'] = None

# specify string for each channel
config['channel_label'] = None
  #0: egfr
  #1: ereg

# whether to not drop the column containing
# pixel
config['drop_pixel_col'] = False

# files to include: all for everyone 
config['include_files'] = 'all' 

## Choice of input features
config['features'] = []

## GT label per loc or per FOV
config['gt_label_scope'] = 'loc'

raise ValueError('This line needs to be changed in line with new set up of gt_label')
## if gt label specified, whih channel is it in (if its label per fov then only one row has to have value), or leave as null
config['gt_label'] = "gt_label"

## specify the gt label map
config['gt_label_map'] = {0: 'unclustered', 1: 'clustered'}

yaml_save_loc = os.path.join(input_folder, 'config/CHANGEME/preprocess.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)

In [ ]:
# Run the preprocess script using this file

!python ../src/locpix_points/scripts/preprocess.py -i ../../../data/nieves/CHANGEME -c ../../../data/nieves/config/CHANGEME/preprocess.yaml -o ../../../output/nieves/CHANGEME -p

### Process data

In [ ]:
# Export config variables 

config = {}

# whether to process the data as heterogeneous or homogenous
# graph
config['hetero'] = False

# train/test/val splits
config['train_ratio'] = 0.7
config['val_ratio'] = 0.1
config['test_ratio'] = 0.2

# what to load into position
# Options: xy, xyz
config['pos']= 'xy'

# what to load into features
# Options: 
config["feat"] = 'xy'
    
# label level
# graph or node
config["label_level"] = 'node'

yaml_save_loc = os.path.join(input_folder, 'config/CHANGEME/process.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)


In [ ]:
# Run the process script 

!python ../src/locpix_points/scripts/process.py -i ../../../output/nieves/CHANGEME -c ../../../data/nieves/config/CHANGEME/process.yaml -r ../../../output/nieves/CHANGEME

### Train

In [ ]:
# Export config variables 

config = {}

# device to train on (gpu or cpu)
config['gpu'] = True

# model parameters
config['model'] = "pointtransformerseg"

# optimiser parameters
config['optimiser'] ="adam"
config['lr'] = 0.001
config['weight_decay'] = 0.0001

# training parameters
config['epochs'] = 5
config['batch_size'] = 1
config['num_workers'] = 1 # generall higher -> faster
config['loss_fn'] = "nll"

config['pointnetseg'] = {
  "k": 16,
  "in_channels": 2,
  "out_channels": 2,
  "dim_model": [32, 64, 128, 256, 512],
  "k_up": 3, # trilinear interpolation
  "output_mlp_layers": 64,
  # ratio of points to sample when transition down
  "ratio": 0.25,
  "pos_nn_layers": 64,
  "attn_nn_layers": 64,
}

# what trying to predict
config["label_level"] = "node" # graph

# train/val transforms
# options: ['normalisescale', 'jitter', 'flip', 'randscale', 'rotate', 'shear'] # null
config["transforms"] = {
    #'jitter': 15, 
    'x_flip': None, 
    'y_flip': None, 
    #'randscale': [0.95, 1.05], 
    'z_rotate': None, 
    #'shear': 0.05, 
    'normalisescale': None
}

# wandb parameters
config["wandb_project"] = "nieves_CHANGEME"
config["wandb_dataset"] = "nieves"

yaml_save_loc = os.path.join(input_folder, 'config/CHANGEME/train.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)

# 1. Choice of augmentations

# 2. Normalise features


In [ ]:
# Run the train script 

!python ../src/locpix_points/scripts/train.py -i ../../../output/nieves/CHANGEME -c ../../../data/nieves/config/CHANGEME/train.yaml

### Evaluate

In [ ]:
# Export config variables

# Run the evaluate script

python src/locpix_points/scripts/evaluate.py -i ../../output/nieves -c src/locpix_points/templates/evaluate.yaml

### Interpret the results

In [ ]:
# Pytorch geometric explainability tools